# Chapter 2 — Routing (Google ADK)

A **Coordinator agent** analyzes incoming requests and delegates to specialist sub-agents:
- **Booker** — handles travel booking requests
- **Info** — handles general knowledge questions

This demonstrates ADK's **Auto-Flow delegation** via `sub_agents`.

In [ ]:
import os
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()

print("Google API Key set:", bool(os.environ.get("GOOGLE_API_KEY")))

## Define Sub-Agents

Each sub-agent is a specialist with its own system instruction.

In [ ]:
from google.adk.agents import Agent

# Specialist: Travel Booker
booker_agent = Agent(
    name="booker",
    model="gemini-2.0-flash",
    description="Handles travel booking requests such as flights, hotels, and itineraries.",
    instruction="You are a travel booking assistant. Help users plan trips, find flights, book hotels, and create itineraries. Be friendly and thorough.",
)

# Specialist: General Info
info_agent = Agent(
    name="info",
    model="gemini-2.0-flash",
    description="Handles general knowledge and informational questions.",
    instruction="You are a helpful general knowledge assistant. Answer questions clearly and concisely with accurate information.",
)

print("Sub-agents defined: booker, info")

## Define the Coordinator (Root Agent)

The coordinator automatically routes requests to the right sub-agent based on the request content.

In [ ]:
coordinator = Agent(
    name="coordinator",
    model="gemini-2.0-flash",
    description="Routes user requests to the appropriate specialist agent.",
    instruction="You are a coordinator. Analyze the user's request and delegate to the appropriate sub-agent. Use 'booker' for travel-related requests and 'info' for general questions.",
    sub_agents=[booker_agent, info_agent],
)

print("Coordinator agent ready with sub_agents:", [a.name for a in coordinator.sub_agents])

## Run the Agent

In [ ]:
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

session_service = InMemorySessionService()
runner = Runner(agent=coordinator, app_name="routing_demo", session_service=session_service)

session = await session_service.create_session(app_name="routing_demo", user_id="user1")

In [ ]:
# Test routing: travel request → should go to booker
travel_request = types.Content(
    role="user",
    parts=[types.Part(text="I want to book a flight from New York to Tokyo next month.")]
)

response = await runner.run_async(user_id="user1", session_id=session.id, new_message=travel_request)

for event in response:
    if event.content and event.content.parts:
        print(f"[{event.author}]: {event.content.parts[0].text}")

In [ ]:
# Test routing: general question → should go to info
info_request = types.Content(
    role="user",
    parts=[types.Part(text="What is the capital of France?")]
)

response = await runner.run_async(user_id="user1", session_id=session.id, new_message=info_request)

for event in response:
    if event.content and event.content.parts:
        print(f"[{event.author}]: {event.content.parts[0].text}")

## Key Takeaways

- **Auto-Flow**: ADK's coordinator automatically delegates based on sub-agent `description` fields
- **`sub_agents`**: The list of agents the coordinator can route to
- **No manual routing logic**: The LLM decides which sub-agent handles each request